In [11]:
import requests
from api_keys import google_maps_key
from api_keys import yelp_key

# Set up the Yelp API endpoint URL
base_url = 'https://api.yelp.com/v3/businesses/search'

# Set up the Google Geocoding API endpoint URL
geocoding_url = 'https://maps.googleapis.com/maps/api/geocode/json'

In [28]:
address = input('Enter an address: ')
#2156 North Claremont Ave, Chicago, IL 60647

Enter an address: 225 43rd St, Sacramento, CA


In [19]:
def fetch_restaurants(latitude, longitude):
    headers = {
        'Authorization': f'Bearer {yelp_key}'
    }

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': 1000,
        'categories': 'restaurants',
        'limit': 50
    }

    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        restaurant_data = response.json()
        return restaurant_data.get('businesses', [])
    else:
        print('Error occurred while fetching restaurant data.')
        return []


In [33]:
def geocode_address(address):
    params = {
        'address': address,
        'key': google_maps_key
    }

    response = requests.get(geocoding_url, params=params)

    if response.status_code == 200:
        geocoding_data = response.json()

        results = geocoding_data.get('results', [])

        if results:
            location = results[0].get('geometry', {}).get('location')
            latitude = location.get('lat')
            longitude = location.get('lng')
            return latitude, longitude
        else:
            return None
    else:
        return None

In [37]:
def restaurant_names(address):
    
    location = geocode_address(address)

    if location:
        latitude, longitude = location
    else:
        return 'Error occurred while geocoding the address.'

    restaurants = fetch_restaurants(latitude, longitude)

    if restaurants:
        restaurant_names = [restaurant.get('name') for restaurant in restaurants]
        count = len(restaurant_names)
        print(f'Total restaurants found: {count}')
        for name in restaurant_names:
            print(name)
    else:
        return 'No restaurants found within the specified radius.'


restaurant_names(address)

Total restaurants found: 20
THAI - The House Of Authentic Ingredients
The Green Room
Selland's Market Cafe
Orphan Breakfast House
Poppy by Mama Kim
Juno's Kitchen & Delicatessen
Casa East Sac
Vibe Health Bar
Cousins Maine Lobster - Sacramento
New Kathmandu Kitchen
Druthers Sutter Park 
Mr Falafel
Midtown Taqueria
Steve’s Pizza
Tiferet Coffee House
Pizza Guys
Calzone Life
Tina's Tacos & Catering
Brown Rice Bakery
J Street Bistro - Mercy General Hospital


In [34]:
def restaurant_types(address):
    location = geocode_address(address)

    if location:
        latitude, longitude = location
    else:
        return 'Error occurred while geocoding the address.'

    restaurants = fetch_restaurants(latitude, longitude)

    if restaurants:
        restaurant_counts = {}
        for restaurant in restaurants:
            categories = restaurant.get('categories', [])
            for category in categories:
                title = category.get('title')
                restaurant_counts[title] = restaurant_counts.get(title, 0) + 1

        sorted_counts = dict(sorted(restaurant_counts.items(), key=lambda x: x[1], reverse=True))

        # Print the restaurant types and counts in descending order
        for category, count in sorted_counts.items():
            print(f'{category}: {count}')

    else:
        return 'No restaurants found.'


restaurant_types(address)

Pizza: 5
Sandwiches: 4
Bars: 2
Cafes: 2
Bakeries: 2
Breakfast & Brunch: 2
Wraps: 2
Seafood: 2
Food Trucks: 2
Coffee & Tea: 2
Thai: 1
Gastropubs: 1
Lounges: 1
Caterers: 1
American (New): 1
Pop-Up Restaurants: 1
Delis: 1
Beer, Wine & Spirits: 1
American (Traditional): 1
Juice Bars & Smoothies: 1
Himalayan/Nepalese: 1
Indian: 1
Waffles: 1
Burgers: 1
Mexican: 1
Salad: 1
Food Delivery Services: 1
Italian: 1
Tacos: 1
Gluten-Free: 1
Vegan: 1
Cafeteria: 1


In [43]:
def avg_rating(address):
    location = geocode_address(address)

    if location:
        latitude, longitude = location
    else:
        return 'Error occurred while geocoding the address.'

    restaurants = fetch_restaurants(latitude, longitude)

    if restaurants:
        total_rating = 0
        total_restaurants = 0

        for restaurant in restaurants:
            rating = restaurant.get('rating')
            if rating:
                total_rating += rating
                total_restaurants += 1

        if total_restaurants > 0:
            avg_rating = total_rating / total_restaurants
            return f'Average rating: {avg_rating}'
        else:
            return 'No ratings available for restaurants in the specified area.'
    else:
        return 'No restaurants found.'

avg_rating(address)

'Average rating: 4.175'